In [1]:
import sys
import torch
import tqdm
import numpy as np
import random
import os
import json
sys.path.append('../../')

In [2]:
from models.cnn.search_cnn import  SearchCNN, SearchCNNController
from models.cnn_var_local.search_cnn  import  LVarSearchCNN, LVarSearchCNNController
from configobj import ConfigObj

In [3]:
import utils
# get data with meta info
input_size, input_channels, n_classes, train_data, valid_data = utils.get_data(
    'mnist', '../../data/', cutout_length=0, validation=True)

train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=64,
                                               shuffle=True,
                                               num_workers=32,
                                               pin_memory=True)

valid_loader = torch.utils.data.DataLoader(valid_data,
                                           batch_size=64,
                                           shuffle=False,
                                           num_workers=1,
                                           pin_memory=True)

/home/local/AP-TEAM/bakhteev/oleg_torch/lib/python3.8/site-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 24, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [38]:
basecfg_path = '../../configs/mnist/var_darts_gamma.cfg'
darts_basecfg_path = '../../configs/mnist/darts.cfg'

cfg = ConfigObj(basecfg_path)
name = cfg['name'] # имя для сохранения результатов
ckp_path = '../../searchs/'+name+'/best_{}.pth.tar' # это шаблон названия сохраненных моделей
seeds = cfg['seeds'].split(';')  # сиды. по идее, можно брать из конфига


fine_tune = True # надо ли дообучать модели
if fine_tune:
    name +='_fine'
name

'mnist_darts_var_gamma_fine'

In [39]:
def var_darts_to_darts(sc):
    sc = sc.to('cpu')
    cfg2 = ConfigObj(darts_basecfg_path)
    cfg2['darts']['sampling_mode'] = 'naive'
    cfg2['device'] = 'cpu'
    sc2 = SearchCNNController(**cfg2)
    
    for a, g in zip(sc2.alpha_reduce, sc.net.q_gamma_reduce):
        a.data *= 0
        a.data += g.data

    
    all_params = {p[0]:p[1] for p in sc2.net.named_parameters()}
    
    for p in sc.net.named_parameters():
        if p[0] in all_params:
            p[1].data *= 0
            p[1].data += all_params[p[0]]
    
    return sc2.cuda()
        
def darts_fix_structure(sc):
    for alpha in sc.alpha_reduce:
        alpha.requires_grad = False
        for subalpha in alpha:
            argm = torch.argmax(subalpha)
            subalpha.data*=0
            subalpha.data[argm] += 1
            
            
            

In [40]:
if fine_tune:    
    cfg = ConfigObj(basecfg_path)    
    cfg['device'] = 'cuda'    
    for s in seeds:        
        sc = LVarSearchCNNController(**cfg)
        sc.load_state_dict(torch.load(ckp_path.format(s)))
        sc.stochastic_gamma = False
        sc.net.stochastic_gamma = False
        for g in sc.net.q_gamma_reduce:
            g.requires_grad = False
            for subgamma in g:
                argm = torch.argmax(subgamma)
                subgamma.data*=0
                subgamma.data[argm] += 1
        sc = sc.to('cuda')
        optim = torch.optim.Adam(sc.all_params)
        correct = 0
        total = 0
        for x,y in tqdm.tqdm(train_loader):
            x = x.cuda()
            y = y.cuda()            
            optim.zero_grad()
            loss = sc.loss(x,y)
            loss.backward()
            optim.step()                              
        torch.save(sc.state_dict(), ckp_path.format(s)+'.fine')

  0%|          | 0/938 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 24, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 938/938 [00:54<00:00, 17.36it/s]


In [41]:
# check everything is ok
cfg = ConfigObj(basecfg_path)
cfg['device'] = 'cuda'
for s in seeds:
    print (s)
    sc = LVarSearchCNNController(**cfg)
    if fine_tune:
        sc.load_state_dict(torch.load(ckp_path.format(s)+'.fine'))
    else:
        sc.load_state_dict(torch.load(ckp_path.format(s)))
    sc = sc.to('cuda')
    sc.eval()
    correct = 0
    total = 0
    for x,y in tqdm.tqdm(valid_loader):
        x = x.cuda()
        y = y.cuda()
        out = sc(x)
        correct += torch.eq(torch.argmax(out, 1), y).sum()
        total += len(x)
    print (correct*1.0/total*1.0)

0


100%|██████████| 157/157 [00:03<00:00, 47.27it/s]


tensor(0.8027, device='cuda:0')
13


100%|██████████| 157/157 [00:03<00:00, 47.87it/s]


tensor(0.7911, device='cuda:0')
21


...

tensor(0.8678, device='cuda:0')
42


100%|██████████| 157/157 [00:03<00:00, 47.79it/s]


tensor(0.7799, device='cuda:0')
99


100%|██████████| 157/157 [00:03<00:00, 47.38it/s]

tensor(0.8326, device='cuda:0')


In [42]:
def var_darts_to_darts(sc):
    sc = sc.to('cpu')
    cfg2 = ConfigObj(darts_basecfg_path)
    cfg2['darts']['sampling_mode'] = 'naive'
    cfg2['device'] = 'cpu'
    sc2 = SearchCNNController(**cfg2)    
    sc2.eval()
    for a, g in zip(sc2.alpha_reduce, sc.net.q_gamma_reduce):
        a.data *= 0
        a.data += g.data        
    
    all_params = sc2.net.state_dict()    
    for p in sc.net.state_dict().items():
        if p[0] in all_params:                        
            all_params[p[0]].data *= 0
            all_params[p[0]].data += p[1]
    
    
    return sc2.cuda()

In [43]:
# check everything is ok, when the structure is fixed
cfg = ConfigObj(basecfg_path)
cfg['device'] = 'cpu'
for s in seeds:
    print (s)
    sc = LVarSearchCNNController(**cfg)
    if fine_tune:
        sc.load_state_dict(torch.load(ckp_path.format(s)+'.fine'))
    else:
        sc.load_state_dict(torch.load(ckp_path.format(s)))
    sc.eval()
    
    sc2 = var_darts_to_darts(sc)
    darts_fix_structure(sc2)
    

    sc2.eval()
    correct = 0
    total = 0
    for x,y in tqdm.tqdm(valid_loader):
        x = x.cuda()
        y = y.cuda()
        out = sc2(x)
        correct += torch.eq(torch.argmax(out, 1), y).sum()
        total += len(x)
    print (correct*1.0/total*1.0)

0


100%|██████████| 157/157 [00:02<00:00, 58.47it/s]


tensor(0.9926, device='cuda:0')
13


100%|██████████| 157/157 [00:02<00:00, 57.13it/s]


tensor(0.9934, device='cuda:0')
21


...

tensor(0.9944, device='cuda:0')
42


100%|██████████| 157/157 [00:02<00:00, 59.30it/s]


tensor(0.9941, device='cuda:0')
99


100%|██████████| 157/157 [00:02<00:00, 60.36it/s]

tensor(0.9947, device='cuda:0')


In [44]:
# FGM
results = []
cfg = ConfigObj(basecfg_path)
cfg['device'] = 'cpu'
for s in seeds:
    results.append([])
    print (s)
    
    sc = LVarSearchCNNController(**cfg)
    if fine_tune:
        sc.load_state_dict(torch.load(ckp_path.format(s)+'.fine'))
    else:
        sc.load_state_dict(torch.load(ckp_path.format(s)))
    sc.eval()
    
    sc2 = var_darts_to_darts(sc)
    darts_fix_structure(sc2)
    

    sc2.eval()
    for eps in np.linspace(0.0, 1.0, 11):
        
        correct = 0
        total = 0

        for x,y in tqdm.tqdm(valid_loader):
            x = x.cuda()
            x.requires_grad = True                                                 
            y = y.cuda()
            out = sc2(x)
            loss = sc2.criterion(out, y)
            sc2.zero_grad()
            loss.backward()
            data_grad = x.grad.data
            sign_data_grad = data_grad.sign()    
            perturbed_image = x + eps*sign_data_grad                    
            out = sc2(perturbed_image)
            correct += torch.eq(torch.argmax(out, 1), y).sum().cpu().detach().numpy()
            total += len(x)
        print (eps, correct*1.0/total*1.0)
        results[-1].append((eps, float(correct*1.0/total*1.0)))
with open(name+'_fgm.json', 'w') as out:
    out.write(json.dumps(results))

0


  0%|          | 0/157 [00:00<?, ?it/s]

0.0 0.9926


  0%|          | 0/157 [00:00<?, ?it/s]

0.1 0.9412


...

0.8 0.0811


  0%|          | 0/157 [00:00<?, ?it/s]

0.9 0.0744


100%|██████████| 157/157 [00:08<00:00, 18.26it/s]

1.0 0.0703


In [45]:
# structure
cfg = ConfigObj(basecfg_path)
cfg['device'] = 'cpu'
for s in seeds:
    results.append([])
    print (s)
    
    sc = LVarSearchCNNController(**cfg)
    if fine_tune:
        sc.load_state_dict(torch.load(ckp_path.format(s)+'.fine'))
    else:
        sc.load_state_dict(torch.load(ckp_path.format(s)))
    sc.eval()
    
    sc2 = var_darts_to_darts(sc)
    darts_fix_structure(sc2)    
    sc2.eval()
    
    ones = []
    for i, alpha in enumerate(sc2.alpha_reduce):
        for j, subalpha in enumerate(alpha):
            ones.append((i,j, torch.argmax(subalpha)))

   
    for eps in np.linspace(0.0, 1.0, 11):
        correct = 0
        total = 0

        for x,y in tqdm.tqdm(valid_loader):
            to_prune = random.sample(ones,  int(len(ones)*eps))
            for i, j, argm in to_prune:
                sc2.alpha_reduce[i][j].data *= 0

            x = x.cuda()                                                          
            y = y.cuda()
            out = sc2(x)            
            correct += torch.eq(torch.argmax(out, 1), y).sum().cpu().detach().numpy()
            total += len(x)
            for i, j, argm in to_prune:
                sc2.alpha_reduce[i][j][argm].data += 1
        print (eps, correct*1.0/total*1.0)
        results.append( (eps, float(correct*1.0/total*1.0)))
with open(name+'_struct.json', 'w') as out:
    out.write(json.dumps(results))

0


  0%|          | 0/157 [00:00<?, ?it/s]

0.0 0.9926


  0%|          | 0/157 [00:00<?, ?it/s]

0.1 0.9829


...

0.8 0.1712


  0%|          | 0/157 [00:00<?, ?it/s]

0.9 0.1327


100%|██████████| 157/157 [00:02<00:00, 59.38it/s]

1.0 0.1009
